### Assumptions/requirements
- 9 categories
- many subfields
- Each judge and project self-identifies both a category and subfield
- Judges stay in the same category througout (this is not strictly necessary, but simplifies the process a lot)
- Thus, the judge-project pairing process works for each category independently.  We will describe this process for a single category.
- The projects (in a single category) should be evenly split among the sessions. If an uneven number, assign fewer to the first session (for late arrivers, like International)
- Judges should not judge posters from their home university.


### Setup tasks
- Determine reasonable lower and upper bounds for proster per session workload for a single judge.
- Determine $Adj$ - the adjacency matrix of subfields.
  - Let $nf$ = number of subfields.  $Adj$ is the $nf \times nf$ matrix where $a_{ij} = 1$ if subfields $i$ and $j$ are adjacent and 0 else.
- Let $P$ be row stochastic version of $A$.  (Sum across each row then divide all entries by their row's sum).  We are doing a simple random walk on the subfield adjancy graph.
- Compute $Dist$ - the distance matrix between subfields.
    1. Initialize $Dist$ as a copy of $Adj$.
    2. Compute $P^2$.  If an entry switches from 0 to non-zero, write 2 in the corresponding entry of $Dist$.  These subfields are 2 jumps apart.
    3. Compute $P^3$.  If an entry switches from 0 to non-zero, write 3 in the corresponding entry of $Dist$.  These subfields are 3 jumps apart.
    4. Continue until (hopefully) all entries of $P^k$ (and thus $Dist$) are non-zero.  $Dist$ is the subfield distance matrix.
- Compute field_incompat - the field incompatibiity matrix
    - field_incompat is the num_judg $\times$ num_post matrix where field_incompat[j,p] = Dist[field_judg[j],field_post[p]].
    - Unless judge[j] and post[p] are from the same university.  In this case, write 999 in that entry.



### Iterative steps

There are 2 major steps: assign posters to sessions AND assign judges to posters

1. Randomly assign posters$\to$sessions such that:
    - All sessions feature the same number of projects if possible.  If not evenly divisible, assign assign one additional poster to each session, starting with the last.  (for late arrivers, like International).
    - We plan to iterate exhaustively over ALL ways to do so.
    - Code is techincal - see inline comments
2. Randomly assign judges$\to$posters such that:
    1. All judges have same workload (#posters to judge)
    2. All posters will receive same number of judgings
    3. Description of algorithm
        1. Use a lottery system where posters select their judges.
        1. Randomize poster selection order. 
        1. Make a copy of big field_incompat with only columns for posters assigned to the session we are looking at.
        1. First poster finds list of ALL judges with minimal field_incompat (aka all rows in its column of field_incompat containing the current min in that column.
        1. Randomly select one of them.
        1. Record that field_incompat score into a second matrix called fi.
        1. Replace that entry in field_incompat with 999.
        1. Increment that judge's workload counter.
        1. If this assignment completes that judge's workload, fill that judge's row with 999
        1. Repeat for all posters in the randomized order selected earlier.  When all all posters have selected once, round 1 of the lottery is done.
        1. Perform more rounds until posters have the desired number of judgings.
        1. Compute field incompatibility score (fis) by summing fi (where we moved field_incompat scores before replacing with 999).  Keep track of best fis found.
        1. Repeat some reasonably large number of times (called num_trials), using the SAME poster$\to$session assignment but new judge$\to$poster assignement
5. Repeat with next poster$\to$session assignment.
6. Repeat for all 9 major categories

In [15]:
from setup import *
from scipy.special import binom

#This is the algorithm for a SINGLE major category.  We will repeat for each category independently.

num_trials = 10 # number of judge->poster assignments analyzed for EACH poster->session assignment
cover = 2 # number of judgings per poster
num_post = 7 # number of posters in this category
num_judg = 5 # number of judges in this category
num_field = 5 # total numder of subfields in ALL categories.  We do not need to split subfields into categories - algorithm handles)
num_ses = 4 #number of sessions
num_univ = 8 #number of universities participating - must enumerate

#########################################################
# Code here picks random values to stand in for real data we will have after registration completes

#Randomly assign judge and poster fields
judg_field = np.random.randint(num_field, size=num_judg)
judg_univ = np.random.randint(num_univ, size=num_judg)
post_field = np.random.randint(num_field, size=num_post)
post_univ = np.random.randint(num_univ, size=num_post)

#Make random field adjency matrix
R = np.random.rand(num_field, num_field)
R = np.triu(R, k=1)
R = (R + R.T)  #must be symmetric
# print(R)
Adj = np.zeros_like(R).astype(int)
Adj[R>.7] = 1
s = Adj.sum(axis=1)
print(s)
P = Adj/(s[:,np.newaxis])
print(P)

#########################################################
# Real code starts here

### Compute field distance matrix
def show():
    print()
    print("k=%d"%k)
    print("P")
    print(P)
    print("Dist")
    print(Dist)
    

Dist = Adj.copy().astype(int)
k = 1
print("Adj")
print(Adj)
show()
for k in range(2,50):
    # Multiply S by itself
    P = P.dot(P)
    # Write k in all entries of Dist that are 0 and the corresponding entry of A just became non-zero
    Dist[((Dist==0) & (P>0))] = k
    if(Dist.min() > 0): #break once Dist is full
        break
    show()
show()
Dist[Dist==0] = 999
Dist[np.diag_indices_from(Dist)] = 0
print(Dist)

# Make field_incompatibility matrix num_judg x num_post, filled with Dist between judge field and poster field
field_incompat = np.fromfunction(lambda j,p: Dist[judg_field[j], post_field[p]], (num_judg,num_post), dtype=int)

# Prevent judge and poster from same univ.  Entry is True is from same univ
univ_match = np.fromfunction(lambda j,p: judg_univ[j] == post_univ[p], (num_judg,num_post), dtype=int)

# Write 999 in corresponding entry of field_incompat
field_incompat[univ_match] = 999
print(field_incompat)


#########################################################
# Compute judge workload per round
q = int(np.floor(num_post/num_ses)) #min number of posters per session
num_post_ses = q*np.ones(num_ses).astype(int) #make vector of length num_ses filled with q
r = num_post - q*num_ses #remainder, if num_ses does not divide num_post evenly
num_post_ses[:r] += 1 #spread these remainders across sessions
num_post_ses = num_post_ses[::-1] #reverse so that first session has fewest (for latecommers)
print(num_post_ses)
workload = np.ceil(num_post_ses.max()*cover/num_judg)
print("Judge workload = %d"%workload)


#########################################################
# Make poster->session assignment generator

def make_all_post_to_ses():
    """
    Generate all possible ways to assign posters to sessions.  Two steps:
    1. Select num_ses posters in increasing order to be the first posters assigned to each ses.
    2. Fill the remaining slots in each ses
    
    Explanation - Step 1 prevents redundancy seen in earlier, more naive versions of this algorithm.
    Example - Suppose num_ses = 2 and num_post = 4.  We do not want to see both
    post_ses = [[0,1],[2,3]] and post_ses = [[2,3],[0,1]].  These are redundant.  
    If you flip ses 0 and ses 1, it is the same (except posters 2&3 present in the earlier session).    
    """

    N = num_post # tracks number of posters not yet assigned to a session
    k = num_ses
    first_post_idx_gen = it.combinations(range(N), k) #generates all ways to select first for each ses in increading order
    N -= k # there are now num_ses fewer poster left to assign
    other_post_idx = []
    for ses in range(num_ses):
        k = num_post_ses[ses]-1 # Number of slots still open in this ses.  Recall - there's already 1 assigned.
        other_post_idx.append(it.combinations(range(N), k)) #generates all ways to fill open slots
        N -= k # fewer posters left to assign
    other_post_idx_gen = it.product(*other_post_idx) #combines the separate generators for each session into one big generator
    return first_post_idx_gen, other_post_idx_gen

#########################################################
# The code below is just a consistency check of the code above.  It computes the total number
# of ways to assign posters to sessions in two distinct ways.  Should produce same answer.
first_post_idx_gen, other_post_idx_gen = make_all_post_to_ses()
s = 0
for _ in first_post_idx_gen:
    s += 1
t = 0
for _ in other_post_idx_gen:
    t += 1
print(s*t)

# count a different way to verify
cs = np.array(num_post_ses) - 1
cs = cs.cumsum().tolist()
cs.insert(0,0)
N = num_post
u = binom(N,num_ses)
N -= num_ses
v = [binom(N-cs[i],num_post_ses[i]-1) for i in range(num_ses)]
print(int(u*np.prod(v)))

#########################################################
#Assign judges to posters
first_post_idx_gen, other_post_idx_gen = make_all_post_to_ses()
best_post_ses = [[999]*n for n in num_post_ses] #Initialize - 999 is sentinel value - mean "this slot not yet filled"
best_judg_post = [[]]*num_ses
best_fi_score_tot = 999999

for first_post_idx in first_post_idx_gen:
    
    for other_post_idx in other_post_idx_gen:
        
        L = list(range(num_post))
        post_ses = [[999]*n for n in num_post_ses] #Initialize - 999 is sentinel value - mean "this slot not yet filled"
        
        # We will walk first_post_idx and other_post_idx backward because we will use the pop method to remove
        # used posters from L.  When an element is popped from L, all elements to its right slide
        # one slot to the left.  But all elements to its left do not move.  Thus, we want to pop from
        # right to left so indexing is consistent.  Example, let L=[0,1,2,3].  let post_idx = [0,2]
        # Thus, we expect to pop 0 and 2 from L.  Suppose we pop 0 first.  Now L = [1,2,3].  When we
        # pop 2, we actually get 3, because 3 is now in slot 2.  However, if we pop 2 first,
        # L = [0,1,2].  Now we pop 0 and get 0.  This is the desired behavior.
        # Recall [::-1] reverses a list        
        for (ses, post_idx) in enumerate(first_post_idx[::-1]):
            # we use -(ses+1) so that the smallest index goes into the first ses and the largest goes into the last ses.
            # We also put into the last slot of each ses (not first slot as discussed above).
            # This is merely a cosmetic preference - makes posters within the same ses appear
            # in a consistent order. But the order of posters within the same ses does not actually matter.
            post_ses[-(ses+1)][-1] = L[post_idx]
            L.pop(post_idx)

        for (ses, post_idx) in enumerate(other_post_idx):
            for (i, j) in enumerate(post_idx[::-1]): # Again, reverse so pop works right (see above)
                post_ses[ses][i] = L.pop(j)
            post_ses[ses] = post_ses[ses][::-1] # Again, cosmetic.  The posters in each session
# are listed in descending order.  This reverses into ascending order.  Turn off to increase speed.
#         print(post_ses)    
    
        #########################################################
        # Assign judges to posters
        best_fi_ses = [[]] * num_ses #best fi matrix
        best_fi_score_ses = [999] * num_ses #best fi_score
        print()
        print("Optimizing poster->session %s"%post_ses)

        for (ses, post) in enumerate(post_ses):  
            print("Optimizing session %d with posters %s"%(ses,post))
            workload = np.ceil(len(post)*cover/num_judg)
#             print(workload)
            for trial in range(num_trials):
                skip_trial = False
                field_incompat_sub = field_incompat[:,post] #Extract cols for posters assigned to this ses
#                 print(field_incompat_sub)
                fi = np.zeros_like(field_incompat_sub)
                judg_work = [0]*num_judg #Tracks each judge's workload
                lot_order = np.random.permutation(len(post)) #randomize poster lottery order
                for i in range(cover): # number of "rounds" of the lottery = cover
                    if(skip_trial == True):
                        break
#                     print("round = %d"%i)
                    for col in lot_order:
#                         print("col = %d"%col)
                        c = field_incompat_sub[:,col] #Grab column for this poster
                        m = c.min()
                        if(m == 999): #If no legal judge remains, abandon this trial
                            skip_trial = True
                            break
                        rows = np.where(c == m)[0] #find all rows in this col where the smallest fi appears
#                         print(rows)
                        if(len(rows) == 1):
                            row = rows[0]
                        else:
                            row = np.random.choice(rows) #choose one such row at random
#                         print(row)
                        fi[row,col] = field_incompat_sub[row,col] #record fi for this pairing
                        judg_work[row] += 1 #increment that judge's workload tracker
                        if(judg_work[row] >= workload):
                            field_incompat_sub[row,:] = 999 #if judge has reached workload limit, fill her row with 999
                        else:
                            field_incompat_sub[row,col] = 999 #else, put 999 in the (row,col) entry only
#                         print(fi)
#                         print(field_incompat_sub)
                if(skip_trial == True):
                    continue
                fi_score = fi.sum() #total fi score for this judge->poster pairing for this ses
#                 print(field_incompat_sub)
#                 print(fi>0)
#                 print(fi)
#                 print(fi_score)
                if(fi_score < best_fi_score_ses[ses]): # if current beats prior best, keep it
                    print("current ses %d fi = %d is better than prior best ses %d fi = %d.  I'll keep it."%(ses,fi_score,ses,best_fi_score_ses[ses]))
                    best_fi_ses[ses] = fi.copy()
                    best_fi_score_ses[ses] = fi_score
                else:
                    print("current ses %d fi = %d is not better than prior best ses %d fi = %d.  I'll ignore it."%(ses,fi_score,ses,best_fi_score_ses[ses]))
#                 print(best_fi_score_ses[ses])
                if(best_fi_score_ses[ses] <= 1):
                    print("Found best possible j->p for ses %d.  Moving on."%ses)        
        print("Finished optimizing session %d"%ses)
        #########################################################
        # We now have the best way to assign judges0>posters under the current poster->session assignment
        # We check if it beats the best from prior poster->session assignment
        print("Finished optimizing poster->session %s"%post_ses)
        fi_score_tot = sum(best_fi_score_ses)
        if(fi_score_tot < best_fi_score_tot):
            print("current total fi = %d is better than prior best total fi = %d.  I'll keep it."%(fi_score_tot,best_fi_score_tot))
            best_post_ses = post_ses.copy()
            best_judg_post = best_fi_ses.copy()
            best_fi_score_tot = fi_score_tot            
            for (ses,(ps,jp)) in enumerate(zip(best_post_ses,best_judg_post)):
                df = pd.DataFrame(jp>0, columns=ps)
                print("session %d"%ses)
                display(df)
                print()
        else:
            print("current total fi = %d is not better than prior best total fi = %d.  I'll ignore it."%(fi_score_tot,best_fi_score_tot))
                

[[ 0.          0.95172283  0.50505158  0.46856326  0.71351086]
 [ 0.          0.          0.48358181  0.97739826  0.67615936]
 [ 0.          0.          0.          0.90121639  0.05429828]
 [ 0.          0.          0.          0.          0.9633711 ]
 [ 0.          0.          0.          0.          0.        ]]
[2 2 1 3 2]
[[ 0.          0.5         0.          0.          0.5       ]
 [ 0.5         0.          0.          0.5         0.        ]
 [ 0.          0.          0.          1.          0.        ]
 [ 0.          0.33333333  0.33333333  0.          0.33333333]
 [ 0.5         0.          0.          0.5         0.        ]]
Adj
[[0 1 0 0 1]
 [1 0 0 1 0]
 [0 0 0 1 0]
 [0 1 1 0 1]
 [1 0 0 1 0]]

k=1
P
[[ 0.          0.5         0.          0.          0.5       ]
 [ 0.5         0.          0.          0.5         0.        ]
 [ 0.          0.          0.          1.          0.        ]
 [ 0.          0.33333333  0.33333333  0.          0.33333333]
 [ 0.5         0.          